In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium
import seaborn as sns
sns.set()

# Import data and remove useless columns 

## Import data

In [2]:
employement_rate = pd.read_csv('./data/labour_market/EMP_RATE_by_birth_place_and_EDUC.csv')
min_wage = pd.read_csv('./data/labour_market/real minimum wage from 2001 to 2019.csv')
avg_duration_unemployment = pd.read_csv('./data/labour_market/average duration of unemployment.csv')
strictness_emply_protec = pd.read_csv('./data/labour_market/strictness of employment protection from 1998 to 2018.csv')
short_time_workers = pd.read_csv('./data/labour_market/percentage of short time workers from 2000.csv')
involuntary_pt_workers = pd.read_csv('./data/labour_market/percentage of involuntary part time workers from 2000.csv')
well_matched_skills_jobs = pd.read_csv('./data/labour_market/percentage of well matched skills for jobs.csv', dtype='unicode')
ft_and_pt_employ = pd.read_csv('./data/labour_market/percentage of full time part time employment.csv')
marginally_attached_workers = pd.read_csv('./data/labour_market/percentage of marginally attached workers from 2000.csv')
public_exp_LMP = pd.read_csv('./data/labour_market/public_exp_LMP.csv')
employees_bargain = pd.read_csv('./data/labour_market/percentage of employees that can bargain from 1998.csv')

## Dictionary with the country code of all OECD countries

In [3]:
oecd_countries = {'AUS': 'Australia', 'AUT': 'Austria', 'BEL': 'Belgium', 'CAN': 'Canada', 'CHL': 'Chile', 'COL': 'Colombia', 'CZE': 'Czech Republic', 'DNK': 'Denmark', 'EST': 'Estonia', 'FIN': 'Finland', 'FRA': 'France', 'DEU': 'Germany', 'GRC': 'Greece', 'HUN': 'Hungary', 'ISL': 'Iceland', 'IRL': 'Ireland', 'ISR': 'Israel', 'ITA': 'Italy', 'JPN': 'Japan', 'KOR': 'Korea', 'LVA': 'Latvia', 'LTU': 'Lithuania', 'LUX': 'Luxembourg', 'MEX': 'Mexico', 'NLD': 'Netherlands', 'NZL': 'New Zealand', 'NOR': 'Norway', 'POL': 'Poland', 'PRT': 'Portugal', 'SVK': 'Slovakia', 'SVN': 'Slovenia', 'ESP': 'Spain', 'SWE': 'Sweden', 'CHE': 'Switzerland', 'TUR': 'Turkey', 'GBR': 'United Kingdom', 'USA': 'United States'}

## Preparation of the data sets

### Removal of unnecessary columns and countries

In [4]:
employement_rate_croped_2015 = employement_rate.drop(['Country', 'Year', 'HATLEV1D', 'Place of birth', 'Reference Period Code', 'Reference Period', 'Flag Codes', 'Flags', 'PowerCode Code', 'PowerCode'], axis=1)
print(employement_rate_croped_2015)
employement_rate_2015_total = employement_rate_croped_2015[employement_rate_croped_2015['Educational attainment']=='Total']
employement_rate_2015_low = employement_rate_croped_2015[employement_rate_croped_2015['Educational attainment']=='Low']
employement_rate_2015_medium = employement_rate_croped_2015[employement_rate_croped_2015['Educational attainment']=='Medium']
employement_rate_2015_high = employement_rate_croped_2015[employement_rate_croped_2015['Educational attainment']=='High']

    COUNTRY BIRTH Educational attainment  YEAR Unit Code        Unit  Value
0       AUS    FB                  Total  2015        PC  Percentage   73.0
1       AUT    FB                  Total  2015        PC  Percentage   67.4
2       AUT    FB                    Low  2015        PC  Percentage   51.3
3       AUT    FB                 Medium  2015        PC  Percentage   71.2
4       AUT    FB                   High  2015        PC  Percentage   75.8
..      ...   ...                    ...   ...       ...         ...    ...
236     TUR    FB                   High  2015        PC  Percentage   60.6
237     TUR    NB                  Total  2015        PC  Percentage   55.2
238     TUR    NB                    Low  2015        PC  Percentage   47.9
239     TUR    NB                 Medium  2015        PC  Percentage   62.5
240     TUR    NB                   High  2015        PC  Percentage   76.7

[241 rows x 7 columns]


### Sorting of data by date